In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle
from tqdm import tqdm
tqdm.pandas()
import statistics
from pubchempy import Compound
from rdkit import Chem, DataStructs
from rdkit.Chem import SaltRemover, QED, rdMolDescriptors
from molvs import Standardizer
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Datasets Task 1: Prepare PubChem Datasets

Author: Kaan Donbekci (donbekci@stanford.edu)

## Contents
* [1.1 Assemble Dataset](#1.1-Assemble-Dataset)
* [1.2 Sanitize Molecules](#1.2-Sanitize-Molecules)
* [1.3 Remove non-druglike molecules](#1.3-Remove-non-druglike-molecules)
* [1.4 Resolve errors](#1.4-Resolve-errors)
    * [1.4.1 Calculate pairwise similarities using fingerprints](#1.4.1-Calculate-pairwise-similarities-using-fingerprints)
    * [1.4.2 Find and remove duplicates](#1.4.2-Find-and-remove-duplicates) # TODO
    * [1.4.3 Find and remove activity cliffs](#1.4.3-Find-and-remove-activity-cliffs) # TODO
* [Exports](#Exports)

In [2]:
plots = False

In [3]:
AVAILABLE_DATASETS = ['ST14', 'KLKB1', 'TMPRSS11D', 'TMPRSS6']

In [4]:
ds_name = 'ST14'

In [5]:
assert ds_name in AVAILABLE_DATASETS

In [6]:
def load_pickle(filename):
    with open(f'../dumps/{filename}.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
IC50_to_Ki_df = pd.read_csv('../data/combined_IC50.csv')

In [8]:
IC50_to_Ki = {}
for i, row in IC50_to_Ki_df.iterrows():
    if row.IC50 == '1':
        IC50_to_Ki[int(row.assay_id)] = float(row.Ki)

## 1.1 Assemble Dataset

First step is to read the dataset as exported from 
[pubchempy](https://pubchem.ncbi.nlm.nih.gov/gene/6768#section=Tested-Compounds&fullscreen=true).


In [9]:
df = pd.read_csv(f'../data/{ds_name}.csv')

Let's drop the rows where the Activity value is not measured in Ki. 

In [10]:
Counter(df.acname)

Counter({'Ki': 417, 'IC50': 20, 'Km': 3, nan: 91, 'Kcat': 3})

In [11]:
def fix_IC50(row):
    if row.aid in IC50_to_Ki and row.acname == 'IC50':
        row.acname = 'Ki'
        row.acvalue = row.acvalue * IC50_to_Ki[row.aid]
    return row

In [12]:
df = df.apply(fix_IC50, axis=1)

In [13]:
df = df[df.acname == 'Ki'].reset_index(drop=True)

Replace categorical labels of unspecified with threshold values

In [14]:
Counter(df.activity)

Counter({'Active': 404, 'Unspecified': 13})

In [15]:
activity_threshold = 50

In [16]:
assert len(df[(df.acvalue < activity_threshold) & (df.activity == 'Inactive')]) == 0
df.loc[(df.acvalue<activity_threshold), 'activity'] = 'Active'

In [17]:
assert len(df[(df.acvalue >= activity_threshold) & (df.activity == 'Active')]) == 0
df.loc[(df.acvalue>=activity_threshold) , 'activity'] = 'Inactive'

Remove rows with nan activity values and unspecified activity

In [18]:
df = df.drop(df[(pd.isna(df.acvalue)) & (df.activity == 'Unspecified')].index)

Some compounds have multiple rows in the dataset, use median activity value and reduce them to a single row.

In [19]:
cid_to_rows = {}
for i, row in df.iterrows():
    if row.cid not in cid_to_rows:
        cid_to_rows[row.cid] = []
    cid_to_rows[row.cid].append(row)

In [20]:
df.activity.unique()

array(['Active', 'Inactive'], dtype=object)

In [21]:
valid_activities = set(['Active', 'Inactive'])

In [22]:
cleaned_rows = []
problem_rows = []
for cid, rows in tqdm(cid_to_rows.items()):
    if len(rows) == 1:
        row = rows[0]
        cleaned_rows.append(row[['cid', 'acvalue', 'activity']])
    else:
        activities = []
        acvalues = []
        for row in rows:
            activities.append(row.activity)
            acvalues.append(row.acvalue)
#             if row.acvalue
        activities = set(activities)
        
        if len(activities) != 1:
            problem_rows.append(rows)
            continue
        activity = activities.pop()
        if activity not in valid_activities:
            continue
#         acvalues = np.array(acvalues)[np.where(pd.notna(acvalues))]
        acvalue = np.nanmedian(acvalues)
        row = pd.Series({'cid': cid, 'acvalue': acvalue, 'activity': activity})
        cleaned_rows.append(row)

100%|██████████| 332/332 [00:00<00:00, 1924.63it/s]


In [23]:
df = pd.DataFrame(cleaned_rows).reset_index(drop=True)

In [24]:
len(df)

332

In [25]:
try:
    cid_to_pubchem = load_pickle(f'{ds_name}_cid_to_pubchem')
except:
    cid_to_pubchem = {}
    print('will send requests to pubchempy, might take a while')

Query pubchempy to get SMILES codes and keep a dictionary of the compounds.

In [26]:
if len(cid_to_pubchem) != len(df):
    for i, row in tqdm(df.iterrows(), total=len(df)):
        if row.cid not in cid_to_pubchem:
            compound = Compound.from_cid(row.cid)
            cid_to_pubchem[row.cid] = compound
assert len(cid_to_pubchem) == len(df)

In [27]:
df['smiles'] = None

In [28]:
def set_smiles(row):
    compound = cid_to_pubchem[row.cid]
    row.smiles = compound.isomeric_smiles
    return row

In [29]:
df = df.progress_apply(set_smiles, axis=1)

100%|██████████| 332/332 [00:00<00:00, 4908.40it/s]


In [30]:
df.head()

,cid,acvalue,activity,smiles
0,44436348,0.05046,Active,CC[C@H](C)[C@H]1C(=O)N[C@H]2CSSC[C@@H](C(=O)N[...
1,11974,0.53500,Active,C1=CC(=C(C=C1C(=N)N)Br)OCCCOC2=C(C=C(C=C2)C(=N...
2,432298,0.20400,Active,C1=CC(=CC=C1C(=N)N)OC2=CC=C(C=C2)OC3=CC=C(C=C3...
3,411406,0.19100,Active,C1=CC(=CC=C1C(=N)N)OCCCCCCOC2=C(C=C(C=C2)C(=N)N)I
4,135435498,4.50000,Active,C1=CC(=CC=C1C(=N)N)N=NC2=C(C=CC3=CC4=C(C=CC(=C...


## 1.2 Sanitize Molecules

Sanitization has two steps: first, standardize the molecule, then, remove the salts from it. We will use RDKit for both tasks.

In [31]:
# EXPORT
cid_to_rdkit = {}

In [32]:
s = Standardizer()
remover = SaltRemover.SaltRemover()
print(f'len(remover.salts) = {len(remover.salts)}')

len(remover.salts) = 15


In [33]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    mol = Chem.MolFromSmiles(row.smiles)
    mol = s.standardize(mol)
    mol = remover.StripMol(mol)
    cid_to_rdkit[row.cid] = mol

100%|██████████| 332/332 [00:01<00:00, 231.21it/s]


## 1.3 Remove non-druglike molecules

In [34]:
property_keys = {'molecular weight': 'MW', 'polar surface area': 'PSA', 'LogP': 'ALOGP', 
                 'rotateable bonds': 'ROTB', 'h-bond donors': 'HBD', 'h-bond acceptors': 'HBA'}

In [35]:
# EXPORT
qed_properties = {key: {} for key in property_keys}

In [36]:
for cid, mol in tqdm(cid_to_rdkit.items()):
    mol_props = QED.properties(mol)
    for key in property_keys:
        qed_properties[key][cid] = mol_props.__getattribute__(property_keys[key])
qed_properties_df = pd.DataFrame(qed_properties)

100%|██████████| 332/332 [00:00<00:00, 413.32it/s]


In [37]:
qed_properties_df.index.name = 'cid'
qed_properties_df.head()

,molecular weight,polar surface area,LogP,rotateable bonds,h-bond donors,h-bond acceptors
cid,,,,,,
44436348,1513.811,549.21,-5.30250,18,17,23
11974,470.165,118.20,3.62764,8,4,4
432298,346.390,118.20,3.83934,6,4,4
411406,480.350,118.20,3.87754,11,4,4
135435498,502.538,189.64,6.80314,6,6,4


In [38]:
if plots:
    for key, prop in qed_properties.items():
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
        fig.suptitle(key)
        sns.boxplot(list(prop.values()), ax=axes[1])
        sns.distplot(list(prop.values()), ax=axes[0])

In [39]:
Q1 = qed_properties_df.quantile(.25)
Q3 = qed_properties_df.quantile(.75)
IQR = Q3 - Q1
threshold = 1.5

In [40]:
qed_properties_outliers_removed_df = qed_properties_df[~(((qed_properties_df < (Q1 - threshold*IQR)) |  (qed_properties_df > (Q3 + threshold*IQR))).any(axis=1))]
print(f'{len(qed_properties_df) - len(qed_properties_outliers_removed_df)} outliers removed.')
qed_properties_df = qed_properties_outliers_removed_df

55 outliers removed.


In [41]:
qed_properties = qed_properties_df.to_dict()
cids_to_keep = list(qed_properties_df.index)

In [42]:
cid_to_rdkit = {cid: cid_to_rdkit[cid] for cid in cids_to_keep}
df = df.query('cid in @cids_to_keep').reset_index(drop=True)

In [43]:
if plots:
    for key, prop in qed_properties.items():
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
        fig.suptitle(f'{key} (w/o outliers)')
        sns.boxplot(list(prop.values()), ax=axes[1])
        sns.distplot(list(prop.values()), ax=axes[0])

## 1.4 Resolve errors

### 1.4.1 Calculate pairwise similarities using fingerprints

In [44]:
N = len(cids_to_keep)

In [45]:
assert (N == len(cid_to_rdkit) and N == len(df))

In [46]:
from functools import partial

In [47]:
fingperint_function = partial(rdMolDescriptors.GetMorganFingerprintAsBitVect, 
                              radius=2, useChirality=True)
fp_name = 'morgan'

In [48]:
# EXPORT
cid_to_fingerprint = {cid: fingperint_function(mol) for cid, mol in cid_to_rdkit.items()
                     }
fingerprint_similarity_matrix = np.empty((N, N))

for i, (cid1, fps1) in tqdm(enumerate(cid_to_fingerprint.items()), total=len(cid_to_fingerprint)):
    for j, (cid2, fps2) in enumerate(cid_to_fingerprint.items()):
        fingerprint_similarity_matrix[i, j] = DataStructs.FingerprintSimilarity(fps1, fps2)

100%|██████████| 277/277 [00:00<00:00, 1098.64it/s]


In [49]:
if plots:
    fig, ax = plt.subplots(figsize=(25,25))
    cax = ax.matshow(fingerprint_similarity_matrix, interpolation='nearest')
    ax.grid(False)
    plt.title('RDKIT fingerprint similarity matrix')
    plt.xticks(range(N), cids_to_keep, rotation=90);
    plt.yticks(range(N), cids_to_keep);
    ax.tick_params(axis='both', which='major', labelsize=4)
    _=fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
    plt.savefig(f'../dumps/{ds_name}_fingerprint_similarity_matrix.png', dpi=400)

In [50]:
if plots:
    cg = sns.clustermap(fingerprint_similarity_matrix, cbar_pos=None, figsize=(15, 15))
    plt.xticks(rotation=90)
    fig = cg.fig
    _ = fig.suptitle('RDKIT fingerprint similarity matrix (clustered)')
    plt.savefig(f'../dumps/{ds_name}_fingerprint_similarity_matrix_clustered', dpi=400)

In [51]:
def fingerprint_to_np(fp):
    bit_string = fp.ToBitString()
    return np.array([int(char) for char in bit_string], dtype=np.uint8)

In [52]:
def add_fingerprint(row):
    row.rdkit_fingerprint = fingerprint_to_np(cid_to_fingerprint[row.cid])
    return row

### 1.4.2 Find and remove duplicates

In [53]:
upper_triangle = (~np.eye(fingerprint_similarity_matrix.shape[0],dtype=bool) * np.triu(fingerprint_similarity_matrix))

In [54]:
similarity_threshold = .90

In [55]:
duplicates = set()
ix_to_cid = {i: key for i, key in enumerate(cid_to_fingerprint.keys())}
while True:
    candidates = {}
    for i, (cid1, fps1) in enumerate(cid_to_fingerprint.items()):
        if i in duplicates: continue
        similar = np.where(upper_triangle[i] > similarity_threshold)[0]
        if len(similar) > 0:
            for j in similar:
                if j in duplicates: continue
                candidates[j] = candidates.get(j, 0) + 1 
                candidates[i] = candidates.get(i, 0) + 1 
    if len(candidates) == 0:
        break
    sorted_candidates = sorted([(val, key) for key, val in candidates.items()], reverse=True)
    duplicates.add(sorted_candidates[0][1])
print('Will remove {} ({:.1%}) compounds.'.format(len(duplicates), len(duplicates)/N))

Will remove 22 (7.9%) compounds.


In [56]:
for i, (cid1, fps1) in enumerate(cid_to_fingerprint.items()):
    assert cid1 == ix_to_cid[i]

In [57]:
duplicates = {ix_to_cid[i] for i in duplicates}

In [58]:
cids_to_keep = list(filter(lambda x: x not in duplicates, cids_to_keep))

In [59]:
df = df.query('cid in @cids_to_keep')

In [60]:
cid_to_rdkit = {cid: cid_to_rdkit[cid] for cid in cids_to_keep}
cid_to_fingerprint = {cid: cid_to_fingerprint[cid] for cid in cids_to_keep}

In [61]:
assert len(cids_to_keep) == len(cid_to_rdkit) == len(df)

### 1.4.3 Find and remove activity cliffs

In [62]:
N = len(df)

In [63]:
# EXPORT
fingerprint_similarity_matrix = np.empty((N, N))

for i, (cid1, fps1) in tqdm(enumerate(cid_to_fingerprint.items()), total=len(cid_to_fingerprint)):
    for j, (cid2, fps2) in enumerate(cid_to_fingerprint.items()):
        fingerprint_similarity_matrix[i, j] = DataStructs.FingerprintSimilarity(fps1, fps2)
upper_triangle = (~np.eye(fingerprint_similarity_matrix.shape[0],dtype=bool) * np.triu(fingerprint_similarity_matrix))

100%|██████████| 255/255 [00:00<00:00, 1202.26it/s]


In [64]:
similarity_threshold = .85
activitiy_ratio_threshold = 100 #this is in folds as in 100-times fold.

In [65]:
cid_to_acvalue = {}
for i, row in df.iterrows():
    cid_to_acvalue[row.cid] = row.acvalue

In [66]:
irregularities = set()
ix_to_cid = {i: key for i, key in enumerate(cid_to_fingerprint.keys())}
for i, (cid1, fps1) in enumerate(cid_to_fingerprint.items()):
    acvalue1 = cid_to_acvalue[cid1]
    similar = np.where(upper_triangle[i] > similarity_threshold)[0]
    if len(similar) > 0:
        for j in similar:
            cid2 = ix_to_cid[j]
            if cid_to_acvalue[cid2] / acvalue1 > activitiy_ratio_threshold or acvalue1 / cid_to_acvalue[cid2] > activitiy_ratio_threshold:
                irregularities.add(cid1)
                irregularities.add(cid2)

print('Will remove {} ({:.1%}) compounds.'.format(len(irregularities), len(irregularities)/N))

Will remove 9 (3.5%) compounds.


In [67]:
cids_to_keep = list(filter(lambda x: x not in irregularities, cids_to_keep))

In [68]:
df = df.query('cid in @cids_to_keep')

In [69]:
cid_to_rdkit = {cid: cid_to_rdkit[cid] for cid in cids_to_keep}
cid_to_fingerprint = {cid: cid_to_fingerprint[cid] for cid in cids_to_keep}

In [70]:
assert len(cids_to_keep) == len(cid_to_rdkit) == len(df)

### 1.4.4 Remove rows with null activity

In [71]:
# downstream should be guaranteed data with non-null activity values
df = df.dropna(subset=['acvalue'])
assert not np.any(np.isnan(df.acvalue))

## Exports & Imports

In [72]:
# remove smiles, add fingerprints, save
df_out = df.drop('smiles', axis=1)  # smiles no longer reflects the processed molecule
df_out['morgan_fingerprint'] = df_out.apply(lambda row: fingerprint_to_np(cid_to_fingerprint[row.cid]), axis=1)
df_out.to_pickle(f'../processed_data/{ds_name}_processed.pkl')

In [73]:
# dump cid_to_pubchem for chaching purposes
# dump cid_to_rdkit for ??? purposes
def save_pickle(obj, filename):
    with open(f'../dumps/{filename}.pkl', 'wb') as f:
        pickle.dump(obj, f)
save_pickle(cid_to_pubchem, f'{ds_name}_cid_to_pubchem')
save_pickle(cid_to_rdkit, f'{ds_name}_cid_to_rdkit')